In [0]:
!mkdir .kaggle
import json
token = {"username":"bgudltaldanny","key":"65b62f1a36c14c834b4e88037463a4f8"}
with open('.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [4]:
!cp .kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6


!kaggle competitions download -c home-depot-product-search-relevance
!unzip home-depot-product-search-relevance.zip -d home-depot-product-search-relevance/
!unzip home-depot-product-search-relevance/train.csv.zip -d home-depot-product-search-relevance/
!unzip home-depot-product-search-relevance/test.csv.zip -d home-depot-product-search-relevance/

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
Processing /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674/kaggle-1.5.6-cp36-none-any.whl
 59% 41.0M/69.4M [00:00<00:00, 32.9MB/s]
100% 69.4M/69.4M [00:00<00:00, 91.8MB/s]
Archive:  home-depot-product-search-relevance.zip
  inflating: home-depot-product-search-relevance/attributes.csv.zip  
  inflating: home-depot-product-search-relevance/product_descriptions.csv.zip  
  inflating: home-depot-product-search-relevance/relevance_instructions.docx  
  inflating: home-depot-product-search-relevance/sample_submission.csv.zip  
  inflating: home-depot-product-search-relevance/test.csv.zip  
  inflating: home-depot-product-search-relevance/train.csv.zip  
Archive:  home-depot-product-search-relevance/train.csv.zip
  inflating: home-depot-product-search-relevance/train.csv  
Archive:  home-depot-product-sea

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout,Conv1D, MaxPool1D,Concatenate
from keras.optimizers import RMSprop
import keras.backend as K

In [0]:
path='./home-depot-product-search-relevance'
train=pd.read_csv(path+'/train.csv',encoding="Latin-1")

In [7]:
train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [0]:
max_product_title_len=train.product_title.map(lambda x: len(x)).max()
max_search_term=train.search_term.map(lambda x: len(x)).max()
max_input_len=max(max_product_title_len,max_search_term)

In [0]:
class MyCharGenerator(object):
  def __init__(self,padding,df,batch_size=32,char_value=0):
    self.padding=padding
    self.char_value=char_value
    self.df=df
    self.batch_size=batch_size


  def __iter__(self):
    while(True):
      X = []
      y = []

      for index,row in self.df.iterrows():
        y.append(row['relevance'])
        st_list= [ord(c) for c in list(row['search_term'])]
        pt_list=[ord(c) for c in list(row['product_title'])]
        X.append((np.pad(st_list,(0,self.padding-len(st_list))),np.pad(pt_list,(0,self.padding-len(pt_list)))))
        # assume there's one document per line, tokens separated by whitespace
        if (index + 1) % self.batch_size == 0:
          X=np.array(X)
          y=np.array(y)
          yield X,y
          X=[]
          y=[]
      
      X=np.array(X)
      y=np.array(y)
      yield X,y

def new_loss(y_actual,y_predicted):
  delta=y_actual*0.15
  diff=K.abs(y_actual-y_predicted)
  ans = K.switch(K.less_equal(diff,delta), lambda: K.square(diff),lambda:1.0- diff)
  return ans

In [153]:
myGen=MyCharGenerator(padding=max_input_len,df=train.head(8),batch_size=1)
it = iter(myGen)
next(it)
# next(it)

(array([[[ 97, 110, 103, 108, 101,  32,  98, 114,  97,  99, 107, 101,
          116,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0],
         [ 83, 105, 109, 112, 115, 111, 110,  32,  83, 116, 114, 111,
          110, 103,  45,  84, 105, 101,  32,  49,  50,  45,  71,

In [0]:
def network(input_shape):
  inp=Input(shape=input_shape)
  x=Conv1D(filters=100,kernel_size=10,activation='relu')(inp)
  x=MaxPool1D()(x)
  x=Conv1D(filters=100,kernel_size=10,activation='relu')(x)
  x=MaxPool1D()(x)
  x=Conv1D(filters=100,kernel_size=10,activation='relu')(x)
  x=Flatten()(x)
  x=Dense(128,activation='relu')(x)
  model= Model(inp,x)

  search_term_input=Input(shape=input_shape)
  product_title_input=Input(shape=input_shape)

  search_term_encoded=model(search_term_input)
  product_title_encoded=model(product_title_input)


  x_connected=Concatenate()([search_term_encoded,product_title_encoded])
  x_connected=Dense(1,activation='sigmoid')(x_connected)

  connected_model=Model(inputs=[search_term_input,product_title_input],output=x_connected)
  

  return connected_model


In [155]:
siames_model=network((max_input_len,1))
siames_model.summary()
siames_model.compile(loss=new_loss ,optimizer='SGD',metrics=['mae'])

Model: "model_60"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_89 (InputLayer)           (None, 147, 1)       0                                            
__________________________________________________________________________________________________
input_90 (InputLayer)           (None, 147, 1)       0                                            
__________________________________________________________________________________________________
model_59 (Model)                (None, 128)          470228      input_89[0][0]                   
                                                                 input_90[0][0]                   
__________________________________________________________________________________________________
concatenate_30 (Concatenate)    (None, 256)          0           model_59[1][0]            

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [156]:
model_hist =siames_model.fit_generator(iter(myGen),verbose=2, steps_per_epoch=1 ,epochs= 3 )

Epoch 1/3


ValueError: ignored